In [3]:
import pandas as pd
import imp
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from scipy.spatial.distance import cdist
from sklearn.model_selection import train_test_split
import nltk
import mlflow

In [4]:
data = pd.read_csv('Womens Clothing E-Commerce Reviews.csv',index_col=0)

In [5]:
data1=data.copy()
data1['review'] = data1['Review Text']
data1['reco'] = data1['Recommended IND']

In [6]:
data1 = data1[['review','reco']]
data1.head()

,review,reco
0,Absolutely wonderful - silky and sexy and comf...,1
1,Love this dress! it's sooo pretty. i happene...,1
2,I had such high hopes for this dress and reall...,0
3,"I love, love, love this jumpsuit. it's fun, fl...",1
4,This shirt is very flattering to all due to th...,1


In [7]:
tf.__version__


'1.10.0'

In [8]:
tf.keras.__version__


'2.1.6-tf'

In [9]:
X = data1['review']
y = data1['reco']

In [10]:
import re
dlist = []
num_words = 10000
tokenizer = Tokenizer(num_words=num_words)
for description in X:
    description=re.sub("[^a-zA-Z]"," ",str(description))
    description = description.lower()
    dlist.append(description)
    
data1['review'] = dlist
data1.head()


,review,reco
0,absolutely wonderful silky and sexy and comf...,1
1,love this dress it s sooo pretty i happene...,1
2,i had such high hopes for this dress and reall...,0
3,i love love love this jumpsuit it s fun fl...,1
4,this shirt is very flattering to all due to th...,1


In [11]:

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.50, 
                                                    random_state=42)

In [12]:
tokenizer.fit_on_texts(X)


In [13]:
if num_words is None:
    num_words = len(tokenizer.word_index)

In [14]:
tokenizer.word_index


{'the': 1,
 'i': 2,
 'it': 3,
 'and': 4,
 'a': 5,
 'is': 6,
 'this': 7,
 'to': 8,
 'in': 9,
 'but': 10,
 'on': 11,
 'for': 12,
 'of': 13,
 'with': 14,
 'was': 15,
 'so': 16,
 'my': 17,
 'dress': 18,
 'that': 19,
 'not': 20,
 's': 21,
 'love': 22,
 't': 23,
 'size': 24,
 'very': 25,
 'have': 26,
 'top': 27,
 'fit': 28,
 'are': 29,
 'like': 30,
 'be': 31,
 'm': 32,
 'as': 33,
 'me': 34,
 'wear': 35,
 'great': 36,
 'too': 37,
 'or': 38,
 'am': 39,
 'you': 40,
 'just': 41,
 'they': 42,
 'would': 43,
 'up': 44,
 'at': 45,
 'fabric': 46,
 'small': 47,
 'color': 48,
 'can': 49,
 'look': 50,
 'if': 51,
 'more': 52,
 'really': 53,
 'ordered': 54,
 'little': 55,
 'perfect': 56,
 'one': 57,
 'will': 58,
 'these': 59,
 'flattering': 60,
 'well': 61,
 'an': 62,
 'soft': 63,
 'out': 64,
 'back': 65,
 'because': 66,
 'had': 67,
 'comfortable': 68,
 'cute': 69,
 'nice': 70,
 'than': 71,
 'bought': 72,
 'beautiful': 73,
 'when': 74,
 'all': 75,
 'looks': 76,
 'bit': 77,
 'fits': 78,
 'large': 79,
 'shi

In [15]:
x_train_tokens = tokenizer.texts_to_sequences(X_train)


In [16]:
X_train[1]


'love this dress   it s sooo pretty   i happened to find it in a store  and i m glad i did bc i never would have ordered it online bc it s petite   i bought a petite and am        i love the length on me  hits just a little below the knee   would definitely be a true midi on someone who is truly petite '

In [17]:
np.array(x_train_tokens[1])


array([ 163,    5,  938,   80,    1, 3534,  209,    6,  340,    4,   42,
         26,    5,   69, 2705,  209,   19,    2,  182,  120,  782,   16,
        555,    9,    1,  106,    1,   28,    6,   47,    2,  319,   44,
        105,   17,  375,    8,    5,   10,  116,   26,  101,  752,    8,
          5,   33,  111,  275,   23,   84, 1063,  312,    1,    4,    1,
        676,    9,   86])

In [18]:
x_test_tokens = tokenizer.texts_to_sequences(X_test)


In [19]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)
np.mean(num_tokens)


58.46351017627523

In [20]:
np.max(num_tokens)


116

In [21]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

119

In [22]:
np.sum(num_tokens < max_tokens) / len(num_tokens)


1.0

In [23]:
pad = 'pre'
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens,
                            padding=pad, truncating=pad)
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)

In [24]:
x_train_pad.shape


(11743, 119)

In [25]:
x_test_pad.shape


(11743, 119)

In [26]:
np.array(x_train_tokens[1])


array([ 163,    5,  938,   80,    1, 3534,  209,    6,  340,    4,   42,
         26,    5,   69, 2705,  209,   19,    2,  182,  120,  782,   16,
        555,    9,    1,  106,    1,   28,    6,   47,    2,  319,   44,
        105,   17,  375,    8,    5,   10,  116,   26,  101,  752,    8,
          5,   33,  111,  275,   23,   84, 1063,  312,    1,    4,    1,
        676,    9,   86])

In [27]:
x_train_pad[1]


array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,  163,    5,  938,   80,    1,
       3534,  209,    6,  340,    4,   42,   26,    5,   69, 2705,  209,
         19,    2,  182,  120,  782,   16,  555,    9,    1,  106,    1,
         28,    6,   47,    2,  319,   44,  105,   17,  375,    8,    5,
         10,  116,   26,  101,  752,    8,    5,   33,  111,  275,   23,
         84, 1063,  312,    1,    4,    1,  676,    9,   86], dtype=int32)

In [28]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

In [29]:
def tokens_to_string(tokens):
    # Map from tokens back to words.
    words = [inverse_map[token] for token in tokens if token != 0]
    
    # Concatenate all words.
    text = " ".join(words)

    return text

In [30]:
X[1]


'love this dress   it s sooo pretty   i happened to find it in a store  and i m glad i did bc i never would have ordered it online bc it s petite   i bought a petite and am        i love the length on me  hits just a little below the knee   would definitely be a true midi on someone who is truly petite '

In [31]:
tokens_to_string(x_train_tokens[1])


'what a sweet shirt the ski pattern is adorable and they have a cute cat pattern that i ve only seen so far in the store the fit is small i sized up from my normal to a but could have even gone to a as there wasn t much difference between the and the except in length'

In [32]:
model = Sequential()
embedding_size = 8
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='layer_embedding'))

/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)


In [33]:
model.add(GRU(units=16, return_sequences=True))


/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)


In [34]:
model.add(GRU(units=8, return_sequences=True))


/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)


In [35]:
model.add(GRU(units=4))


/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)


In [36]:
model.add(Dense(1, activation='sigmoid'))


/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)


In [37]:
optimizer = Adam(lr=1e-3)


In [38]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [39]:
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 119, 8)            80000     
_________________________________________________________________
gru (GRU)                    (None, 119, 16)           1200      
_________________________________________________________________
gru_1 (GRU)                  (None, 119, 8)            600       
_________________________________________________________________
gru_2 (GRU)                  (None, 4)                 156       
_________________________________________________________________
dense (Dense)                (None, 1)                 5         
Total params: 81,961
Trainable params: 81,961
Non-trainable params: 0
_________________________________________________________________


In [40]:
model.fit(x_train_pad, y_train,
          validation_split=0.05, epochs=10, batch_size=64)

/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)


Train on 11155 samples, validate on 588 samples
Epoch 1/10
11155/11155 [==============================] - 71s 6ms/step - loss: 0.4809 - acc: 0.8238 - val_loss: 0.4691 - val_acc: 0.8197
Epoch 2/10
11155/11155 [==============================] - 68s 6ms/step - loss: 0.4592 - acc: 0.8238 - val_loss: 0.4553 - val_acc: 0.8197
Epoch 3/10
11155/11155 [==============================] - 69s 6ms/step - loss: 0.4230 - acc: 0.8258 - val_loss: 0.3716 - val_acc: 0.8418
Epoch 4/10
11155/11155 [==============================] - 71s 6ms/step - loss: 0.3375 - acc: 0.8577 - val_loss: 0.3129 - val_acc: 0.8673
Epoch 5/10
11155/11155 [==============================] - 73s 7ms/step - loss: 0.2801 - acc: 0.8851 - val_loss: 0.2769 - val_acc: 0.8793
Epoch 6/10
11155/11155 [==============================] - 83s 7ms/step - loss: 0.2484 - acc: 0.9037 - val_loss: 0.2574 - val_acc: 0.9014
Epoch 7/10
11155/11155 [==============================] - 74s 7ms/step - loss: 0.2227 - acc: 0.9135 - val_loss: 0.2792 - val_acc: 

In [41]:
result = model.evaluate(x_test_pad, y_test)

11743/11743 [==============================] - 30s 3ms/step


In [42]:
print("Accuracy: {0:.2%}".format(result[1]))


Accuracy: 88.46%


In [43]:
y_pred = model.predict(x=x_test_pad[0:1000])
y_pred = y_pred.T[0]

/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:75: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  return _inspect.getargspec(target)


In [44]:
cls_pred = np.array([1.0 if p>0.5 else 0.0 for p in y_pred])


In [45]:
cls_true = np.array(y_test[0:1000])


In [46]:
incorrect = np.where(cls_pred != cls_true)
incorrect = incorrect[0]

In [47]:
len(incorrect)


127

In [48]:
idx = incorrect[0]
idx

14

In [49]:
text = X[idx]
text

'this is a nice choice for holiday gatherings  i like that the length grazes the knee so it is conservative enough for office related gatherings  the size small fit me well   i am usually a size     with a small bust  in my opinion it runs small and those with larger busts will definitely have to size up  but then perhaps the waist will be too big   the problem with this dress is the quality  the fabrics are terrible  the delicate netting type fabric on the top layer of skirt got stuck in the zip'

In [50]:
y_pred[idx]


0.66259795

In [51]:
cls_true[idx]


0

In [52]:
text1 = 'i had such high hopes for this dress and really wanted it to work for me  i initially ordered the petite small  my usual size  but i found this to be outrageously small  so small in fact that i could not zip it up  i reordered it in petite medium  which was just ok  overall  the top half was comfortable and fit nicely  but the bottom half had a very tight under layer and several somewhat cheap  net  over layers  imo  a major design flaw was the net over layer sewn directly into the zipper   it c'
text2 = "Good shoes!"
text3 = "Maybe I like this dress."
text4 = "Meh ..."
text5 = "If I were a drunk teenager then this shoes might be good."
text6 = "wonderful dress!"
text7 =  " bad dress!"
text8 = "This dress really sucks! Can I get my money back please?"
texts = [text1, text2, text3, text4, text5, text6, text7, text8]


In [53]:
tokens = tokenizer.texts_to_sequences(texts)


In [54]:
tokens_pad = pad_sequences(tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)
tokens_pad.shape

(8, 119)

In [55]:
model.predict(tokens_pad)


array([[0.2028821 ],
       [0.97472477],
       [0.70829415],
       [0.91254777],
       [0.33958483],
       [0.96855867],
       [0.5102551 ],
       [0.4076098 ]], dtype=float32)